# Performance Check

In [2]:
import timeit
import numpy as np
from npgeohash import npgeohash, npgeohash_jit

In [3]:
poi = np.array(
    [
        [35.69240645093, 139.7034750767164],
        [35.691255831981294, 139.69025228754268],
        [35.68307317410146, 139.71622562341963],
        [35.70071598380815, 139.69749333875686],
        [35.68997734701496, 139.6847427920536],
        [35.68115524225217, 139.68584469115146],
    ]
)
lat, lon = 35.68952987243547, 139.69953972279566

results = {}


def _t(stmt):
    t = timeit.Timer(stmt, globals=globals())
    n = 10000
    u = t.timeit(n) / n * 1e6
    print(f"{u:.2f} us")
    results[stmt] = u

The following function is jitted.

In [4]:
_t("npgeohash.encode_array(poi, 7)")
_t("npgeohash_jit.encode_array(poi, 7)")

49.71 us
9.43 us


In [5]:
_t("list(npgeohash.create_circle(lat, lon, 1000, 7))")
_t("list(npgeohash_jit.create_circle(lat, lon, 1000, 7))")

1244.19 us
279.64 us


In [6]:
poi_geohashes = npgeohash.encode_array(poi, 7)

_t("npgeohash.neighbors(poi_geohashes[0])")
_t("npgeohash_jit.neighbors(poi_geohashes[0])")

54.26 us
15.79 us


The following functions are actually not jitted.

In [7]:
_t("npgeohash.many_neighbors(poi_geohashes)")
_t("npgeohash_jit.many_neighbors(poi_geohashes)")

331.00 us
338.71 us


In [8]:
_t("npgeohash.isin(poi_geohashes, poi_geohashes)")
_t("npgeohash_jit.isin(poi_geohashes, poi_geohashes)")

34.93 us
34.31 us


In [9]:
_t("npgeohash.isin_circle(poi_geohashes, lat, lon, 1000, 7)")
_t("npgeohash_jit.isin_circle(poi_geohashes, lat, lon, 1000, 7)")

2374.09 us
2378.97 us


# Result

In [10]:
from IPython.core.display import Markdown

r_nojit = {}
r_jit = {}
for func, time in results.items():
    i = func.find("npgeohash")
    j = func.find(".", i)
    k = func.find("(", j)
    module_name = func[i:j]
    func_name = func[j + 1:k]
    if module_name == "npgeohash_jit":
        r_jit[func_name] = time
    else:
        r_nojit[func_name] = time
table = ["Function|npgeohash_jit(w/ jit)|npgeohash(wo/ jit)|Times", "---|---|---|---"]
for func_name in r_nojit.keys():
    table.append(f"{func_name}|{r_jit[func_name]:.2f}|{r_nojit[func_name]:.2f}|{r_nojit[func_name] / r_jit[func_name]:.2f}")
display(Markdown("\n".join(table)))
    

Function|npgeohash_jit(w/ jit)|npgeohash(wo/ jit)|Times
---|---|---|---
encode_array|9.43|49.71|5.27
create_circle|279.64|1244.19|4.45
neighbors|15.79|54.26|3.44
many_neighbors|338.71|331.00|0.98
isin|34.31|34.93|1.02
isin_circle|2378.97|2374.09|1.00